# Example of usage lambda-calculus library from pip

In [1]:
from lambda_calculus import Variable
from lambda_calculus import Abstraction
from lambda_calculus import Application

from lambda_calculus.visitors.normalisation import BetaNormalisingVisitor

## Nesting

In [2]:
term = Application(Variable("+"), Variable("x"))
term = Application(term, Variable("y"))
term = Abstraction("y", term)
term = Abstraction("x", term)
term = Application(term, Variable("y"))
term = Application(term, Variable("3"))
term = Abstraction("y", term)
term = Application(term, Variable("4"))

print(term)

((λy.(((λx.(λy.((+ x) y))) y) 3)) 4)


## Utility Methods

In [3]:
x = Variable.with_valid_name("x")
y = Variable.with_valid_name("y")

term = Application.with_arguments(Variable.with_valid_name("+"), (x, y))
term = Abstraction.curried(("x", "y"), term)
term = Application.with_arguments(term, (y, Variable.with_valid_name("3")))
term = Abstraction("y", term)
term = Application(term, Variable.with_valid_name("4"))

print(term)

((λy.(((λx.(λy.((+ x) y))) y) 3)) 4)


## Method Chaining

In [4]:
x = Variable.with_valid_name("x")
y = Variable.with_valid_name("y")

term = (
    Variable("+")
    .apply_to(x, y)
    .abstract("x", "y")
    .apply_to(y, Variable("3"))
    .abstract("y")
    .apply_to(Variable("4"))
)
print(term)

((λy.(((λx.(λy.((+ x) y))) y) 3)) 4)


## Evaluation

In [5]:
assert BetaNormalisingVisitor().skip_intermediate(term) == Application.with_arguments(
    Variable("+"), (Variable("4"), Variable("3"))
)

In [6]:
BetaNormalisingVisitor().skip_intermediate(term)

Application(abstraction=Application(abstraction=Variable(name='+'), argument=Variable(name='4')), argument=Variable(name='3'))

In [7]:
Application.with_arguments(Variable("+"), (Variable("4"), Variable("3")))

Application(abstraction=Application(abstraction=Variable(name='+'), argument=Variable(name='4')), argument=Variable(name='3'))

# Default lambdas in Python example

In [8]:
import functools

result = functools.reduce(lambda x, y: x + y, [1, 2, 3, 4, 5])

print(result)

15


# More tests of lambda-calculus lib

In [28]:
from lambda_calculus.terms import logic
from lambda_calculus.terms import arithmetic
from lambda_calculus.terms import pairs
from lambda_calculus.terms import combinators

In [10]:
logic.TRUE

Abstraction(bound='x', body=Abstraction(bound='y', body=Variable(name='x')))

In [12]:
term_or_true_false = Application(Application(logic.OR, logic.TRUE), logic.FALSE)

In [14]:
BetaNormalisingVisitor().skip_intermediate(term_or_true_false)

Abstraction(bound='x', body=Abstraction(bound='y', body=Variable(name='x')))

# REALIZE NUMBERS

In [23]:
def n0_term():
    return Abstraction("s", Abstraction("z", Variable("z")))


n0_term()

Abstraction(bound='s', body=Abstraction(bound='z', body=Variable(name='z')))

In [24]:
def n_term(n: int):
    if n < 0:
        raise ValueError("in lambda calculus number can't be less than 0")
    if n == 0:
        return n0_term()
    core_term = Application(Variable("s"), Variable("z"))
    for _ in range(n - 1):
        core_term = Application(Variable("s"), core_term)
    return Abstraction("s", Abstraction("z", core_term))

In [25]:
n_term(0)

Abstraction(bound='s', body=Abstraction(bound='z', body=Variable(name='z')))

In [26]:
n_term(1)

Abstraction(bound='s', body=Abstraction(bound='z', body=Application(abstraction=Variable(name='s'), argument=Variable(name='z'))))

In [27]:
n_term(2)

Abstraction(bound='s', body=Abstraction(bound='z', body=Application(abstraction=Variable(name='s'), argument=Application(abstraction=Variable(name='s'), argument=Variable(name='z')))))

# TEST NUMBERS

### test ISZERO

In [29]:
term = Application(arithmetic.ISZERO, n_term(0))
print(term, "\n\n")
print(BetaNormalisingVisitor().skip_intermediate(term))

((λn.((n (λx.(λx.(λy.y)))) (λx.(λy.x)))) (λs.(λz.z))) 


(λx.(λy.x))


In [30]:
term = Application(arithmetic.ISZERO, n_term(1))
print(term, "\n\n")
print(BetaNormalisingVisitor().skip_intermediate(term))

((λn.((n (λx.(λx.(λy.y)))) (λx.(λy.x)))) (λs.(λz.(s z)))) 


(λx.(λy.y))


In [31]:
term = Application(arithmetic.ISZERO, n_term(100))
# print(term, "\n\n")
print(BetaNormalisingVisitor().skip_intermediate(term))

(λx.(λy.y))


### test SUCC

In [32]:
term = Application(arithmetic.SUCCESSOR, n_term(0))
print(term, "\n\n")
print(BetaNormalisingVisitor().skip_intermediate(term))

((λn.(λf.(λx.(f ((n f) x))))) (λs.(λz.z))) 


(λf.(λx.(f x)))


In [33]:
term = Application(arithmetic.SUCCESSOR, n_term(1))
print(term, "\n\n")
print(BetaNormalisingVisitor().skip_intermediate(term))

((λn.(λf.(λx.(f ((n f) x))))) (λs.(λz.(s z)))) 


(λf.(λx.(f (f x))))


In [34]:
term = Application(arithmetic.SUCCESSOR, n_term(2))
print(term, "\n\n")
print(BetaNormalisingVisitor().skip_intermediate(term))

((λn.(λf.(λx.(f ((n f) x))))) (λs.(λz.(s (s z))))) 


(λf.(λx.(f (f (f x)))))


In [35]:
term = Application(Application(pairs.PAIR, arithmetic.number(0)), arithmetic.number(2))

f_term = Application(pairs.FIRST, term)
s_term = Application(pairs.SECOND, term)

print(BetaNormalisingVisitor().skip_intermediate(f_term))
print(BetaNormalisingVisitor().skip_intermediate(s_term))

(λf.(λx.x))
(λf.(λx.(f (f x))))


# creating and testing SINC term

In [36]:
def sinc_term():
    p = Variable("p")
    term_inner_0 = Application(pairs.SECOND, p)
    term_inner_1 = Application(arithmetic.SUCCESSOR, Application(pairs.SECOND, p))
    app = Application(Abstraction(pairs.PAIR, term_inner_0), term_inner_1)
    return Abstraction("p", app)

In [37]:
s_term = Application(sinc_term(), term)
print(term, "\n\n")
print(BetaNormalisingVisitor().skip_intermediate(term))

(((λx.(λy.(λf.((f x) y)))) (λf.(λx.x))) (λf.(λx.(f (f x))))) 


(λf.((f (λf.(λx.x))) (λf.(λx.(f (f x))))))


In [38]:
def pred_term():
    n = Variable("n")
    term_inner_0 = Application(
        Application(n, sinc_term()),
        Application(
            Application(pairs.PAIR, arithmetic.number(0)), arithmetic.number(0)
        ),
    )
    return Abstraction("n", Application(pairs.FIRST, term_inner_0))

In [39]:
def substr_term():
    m = Variable("m")
    n = Variable("n")
    return Abstraction(
        "n", Abstraction("m", Application(Abstraction(m, pred_term()), n))
    )

In [40]:
def le_term():
    m = Variable("m")
    n = Variable("n")
    return Abstraction(
        "n",
        Abstraction(
            "m",
            Application(
                arithmetic.ISZERO, Application(Application(substr_term(), n), m)
            ),
        ),
    )

In [41]:
def eq_term():
    m = Variable("m")
    n = Variable("n")
    term_le_0 = Application(Application(le_term(), n), m)
    term_le_1 = Application(Application(le_term(), m), n)
    return Abstraction(
        "n", Abstraction("m", Application(Application(logic.AND, term_le_0), term_le_1))
    )

In [42]:
def gcd_term():
    n, m, f = Variable("n"), Variable("m"), Variable("f")
    sub_m_n = Application(Application(substr_term(), m), n)
    sub_n_m = Application(Application(substr_term(), n), m)
    iszero_n = Application(arithmetic.ISZERO, n)
    iszero_m = Application(arithmetic.ISZERO, m)
    f_n_sub = Application(Application(f, n), sub_m_n)
    f_m_sub = Application(Application(f, m), sub_n_m)
    le_n_m = Application(Application(le_term(), n), m)
    eq_n_m = Application(Application(eq_term(), n), m)
    or_is_zero = Application(logic.OR, Application(iszero_n, iszero_m))
    ite_le_sub_sub = Application(
        Application(Application(logic.IF_THEN_ELSE, le_n_m), f_n_sub), f_m_sub
    )
    ite_eq_ite = Application(
        Application(Application(logic.IF_THEN_ELSE, eq_n_m), n), ite_le_sub_sub
    )
    or_is_ite = Application(Application(or_is_zero, arithmetic.number(0)), ite_eq_ite)
    inner_app = Application(logic.IF_THEN_ELSE, or_is_ite)
    inner_lambda = Abstraction("f", Abstraction("n", Abstraction("m", inner_app)))
    return Application(combinators.Y, inner_lambda)

# testing GCD function:

In [43]:
print("\n\nGCD N0 N0 == 0")
term = Application(Application(gcd_term(), arithmetic.number(0)), arithmetic.number(0))
print(BetaNormalisingVisitor().skip_intermediate(term))



GCD N0 N0 == 0
(λa.(λb.b))


In [44]:
first_num = 1
second_num = 0
print(f"\n\nGCD N_{first_num} N_{second_num} == 0")
term = Application(
    Application(gcd_term(), arithmetic.number(first_num)), arithmetic.number(second_num)
)
print(BetaNormalisingVisitor().skip_intermediate(term))



GCD N_1 N_0 == 0
(λa.(λb.(a b)))


In [45]:
first_num = 0
second_num = 1
print(f"\n\nGCD N_{first_num} N_{second_num} == 0")
term = Application(
    Application(gcd_term(), arithmetic.number(first_num)), arithmetic.number(second_num)
)
print(BetaNormalisingVisitor().skip_intermediate(term))



GCD N_0 N_1 == 0
(λa.(λb.a))


In [46]:
first_num = 1
second_num = 1
print(f"\n\nGCD N_{first_num} N_{second_num} == 0")
term = Application(
    Application(gcd_term(), arithmetic.number(first_num)), arithmetic.number(second_num)
)
print(BetaNormalisingVisitor().skip_intermediate(term))



GCD N_1 N_1 == 0
(λa.(λb.(a b)))


In [47]:
first_num = 3
second_num = 1
print(f"\n\nGCD N_{first_num} N_{second_num} == 0")
term = Application(
    Application(gcd_term(), arithmetic.number(first_num)), arithmetic.number(second_num)
)
print(BetaNormalisingVisitor().skip_intermediate(term))



GCD N_3 N_1 == 0
(λa.(λb.(a b)))


In [48]:
first_num = 3
second_num = 3
print(f"\n\nGCD N_{first_num} N_{second_num} == 0")
term = Application(
    Application(gcd_term(), arithmetic.number(first_num)), arithmetic.number(second_num)
)
print(BetaNormalisingVisitor().skip_intermediate(term))



GCD N_3 N_3 == 0
(λa.(λb.(a b)))


In [49]:
first_num = 4
second_num = 1
print(f"\n\nGCD N_{first_num} N_{second_num} == 0")
term = Application(
    Application(gcd_term(), arithmetic.number(first_num)), arithmetic.number(second_num)
)
print(BetaNormalisingVisitor().skip_intermediate(term))



GCD N_4 N_1 == 0
(λa.(λb.(a b)))


In [50]:
first_num = 40
second_num = 20
print(f"\n\nGCD N_{first_num} N_{second_num} == 0")
term = Application(
    Application(gcd_term(), arithmetic.number(first_num)), arithmetic.number(second_num)
)
print(BetaNormalisingVisitor().skip_intermediate(term))



GCD N_40 N_20 == 0
(λa.(λb.(a b)))
